# Advanced Deep Learning with Keras

## 1. The Keras Functional API


In [1]:
# Import Input from tensorflow.keras.layers
from tensorflow.keras.layers import Input

# Create an input layer of shape 1
input_tensor = Input(shape=(1,))

In [2]:
# Load layers
from tensorflow.keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Dense layer
output_layer = Dense(1)

# Connect the dense layer to the input_tensor
output_tensor = output_layer(input_tensor)

In [3]:
# Load layers
from tensorflow.keras.layers import Input, Dense

# Input layer
input_tensor = Input(shape=(1,))

# Create a dense layer and connect the dense layer to the input_tensor in one step
# Note that we did this in 2 steps in the previous exercise, but are doing it in one step now
output_tensor = Dense(1)(input_tensor)

In [4]:
# Input/dense/output layers
from tensorflow.keras.layers import Input, Dense
input_tensor = Input(shape=(1,))
output_tensor = Dense(1)(input_tensor)

# Build the model
from tensorflow.keras.models import Model
model = Model(input_tensor, output_tensor)

In [5]:
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [6]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install graphviz 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import the plotting function
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Summarize the model
model.summary()

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

games_tourney = pd.read_csv('games_tourney.csv')
games_tourney_train, games_tourney_test = train_test_split(games_tourney, test_size=0.19, random_state=42)

In [9]:
# Now fit the model
model.fit(games_tourney_train['seed_diff'], games_tourney_train['score_diff'],
          epochs=1,
          batch_size=128,
          validation_split=0.1,
          verbose=True)

25/25 [==============================] - 1s 8ms/step - loss: 12.8051 - val_loss: 11.7803


In [10]:
# Load the X variable from the test data
X_test = games_tourney_test['seed_diff']

# Load the y variable from the test data
y_test = games_tourney_test['score_diff']

# Evaluate the model on the test data
print(model.evaluate(X_test, y_test, verbose=False))

12.856620788574219


## 2. Two Input Networks Using Categorical Embeddings, Shared Layers, and Merge Layers


In [11]:
import pandas as pd

games_season = pd.read_csv('games_season.csv')
games_season

,season,team_1,team_2,home,score_diff,score_1,score_2,won
0,1985,3745,6664,0,17,81,64,1
1,1985,126,7493,1,7,77,70,1
2,1985,288,3593,1,7,63,56,1
3,1985,1846,9881,1,16,70,54,1
4,1985,2675,10298,1,12,86,74,1
...,...,...,...,...,...,...,...,...
312173,2018,3592,1481,1,-1,55,56,0
312174,2018,7322,2081,1,-1,57,58,0
312175,2018,9730,3233,1,-13,61,74,0
312176,2018,8813,4297,1,-5,72,77,0


In [12]:
# Imports
from tensorflow.keras.layers import Embedding
from numpy import unique

# Count the unique number of teams
n_teams = unique(games_season['team_1']).shape[0]

# Create an embedding layer
team_lookup = Embedding(input_dim=n_teams,
                        output_dim=1,
                        input_length=1,
                        name='Team-Strength')

In [13]:
# Imports
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model

# Create an input layer for the team ID
teamid_in = Input(shape=(1,))

# Lookup the input in the team strength embedding layer
strength_lookup = team_lookup(teamid_in)

# Flatten the output
strength_lookup_flat = Flatten()(strength_lookup)

# Combine the operations into a single, re-usable model
team_strength_model = Model(teamid_in, strength_lookup_flat, name='Team-Strength-Model')

In [14]:
# Load the input layer from tensorflow.keras.layers
from tensorflow.keras.layers import Input

# Input layer for team 1
team_in_1 = Input((1,), name='Team-1-In')

# Separate input layer for team 2
team_in_2 = Input((1,), name='Team-2-In')

In [15]:
# Lookup team 1 in the team strength model
team_1_strength = team_strength_model(team_in_1)

# Lookup team 2 in the team strength model
team_2_strength = team_strength_model(team_in_2)

In [16]:
# Import the Subtract layer from tensorflow.keras
from tensorflow.keras.layers import Subtract

# Create a subtract layer using the inputs from the previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

In [17]:
# Imports
from tensorflow.keras.layers import Subtract
from tensorflow.keras.models import Model

# Subtraction layer from previous exercise
score_diff = Subtract()([team_1_strength, team_2_strength])

# Create the model
model = Model([team_in_1, team_in_2], score_diff)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [18]:
# Get the team_1 column from the regular season data
input_1 = games_season['team_1']

# Get the team_2 column from the regular season data
input_2 = games_season['team_2']

# Fit the model to input 1 and 2, using score diff as a target
model.fit([input_1, input_2],
          games_season['score_diff'],
          epochs=1,
          batch_size=2048,
          validation_split=0.1,
          verbose=True)

138/138 [==============================] - 1s 3ms/step - loss: 12.1204 - val_loss: 11.8385


In [19]:
# Get team_1 from the tournament data
input_1 = games_tourney['team_1']

# Get team_2 from the tournament data
input_2 = games_tourney['team_2']

# Evaluate the model using these inputs
print(model.evaluate([input_1, input_2], games_tourney['score_diff'], verbose=False))

11.681172370910645


## 3. Multiple Inputs: 3 Inputs (and Beyond!)


In [20]:
from tensorflow.keras.layers import Concatenate

# Create an Input for each team
team_in_1 = Input(shape=(1,), name='Team-1-In')
team_in_2 = Input(shape=(1,), name='Team-2-In')

# Create an input for home vs away
home_in = Input(shape=(1,), name='Home-In')

# Lookup the team inputs in the team strength model
team_1_strength = team_strength_model(team_in_1)
team_2_strength = team_strength_model(team_in_2)

# Combine the team strengths with the home input using a Concatenate layer, then add a Dense layer
out = Concatenate()([team_1_strength, team_2_strength, home_in])
out = Dense(1)(out)

In [21]:
# Import the model class
from tensorflow.keras.models import Model

# Make a Model
model = Model([team_in_1, team_in_2, home_in], out)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [22]:
# Fit the model to the games_season dataset
model.fit([games_season['team_1'], games_season['team_2'], games_season['home']],
          games_season['score_diff'],
          epochs=1,
          verbose=True,
          validation_split=0.1,
          batch_size=2048)

# Evaluate the model on the games_tourney dataset
print(model.evaluate([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']], 
                     games_tourney['score_diff'], verbose=False))

138/138 [==============================] - 1s 3ms/step - loss: 12.1774 - val_loss: 11.8428
11.684207916259766


In [23]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Team-1-In (InputLayer)      [(None, 1)]                  0         []                            
                                                                                                  
 Team-2-In (InputLayer)      [(None, 1)]                  0         []                            
                                                                                                  
 Team-Strength-Model (Funct  (None, 1)                    10888     ['Team-1-In[0][0]',           
 ional)                                                              'Team-2-In[0][0]']           
                                                                                                  
 Home-In (InputLayer)        [(None, 1)]                  0         []                      

Answer: 10,892

Answer: 10,892

Answer: Team-Strength (Model)

In [ ]:
# Imports
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

Answer: 3

Answer: 1

Answer: Team-Strength

In [24]:
# Predict
games_tourney['pred'] = model.predict([games_tourney['team_1'], games_tourney['team_2'], games_tourney['home']])

133/133 [==============================] - 0s 2ms/step


In [25]:
games_tourney_train, games_tourney_test = train_test_split(games_tourney, test_size=0.19, random_state=42)

In [26]:
# Create an input layer with 3 columns
input_tensor = Input((3,))

# Pass it to a Dense layer with 1 unit
output_tensor = Dense(1)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [27]:
# Fit the model
model.fit(games_tourney_train[['home', 'seed_diff', 'pred']],
          games_tourney_train['score_diff'],
          epochs=1,
          verbose=True)

108/108 [==============================] - 0s 1ms/step - loss: 16.4852


In [29]:
# Evaluate the model on the games_tourney_test dataset
print(model.evaluate(games_tourney_test[['home', 'seed_diff', 'pred']],
                     games_tourney_test['score_diff'], verbose=False))

16.004404067993164


## 4. Multiple Outputs


In [30]:
# Define the input
input_tensor = Input((2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Create a model
model = Model(input_tensor, output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

In [31]:
# Fit the model
model.fit(games_tourney_train[['seed_diff', 'pred']],
          games_tourney_train[['score_1', 'score_2']],
          verbose=True,
          epochs=100,
          batch_size=16384)

Epoch 1/100
1/1 [==============================] - 0s 334ms/step - loss: 70.9587
Epoch 2/100
1/1 [==============================] - 0s 6ms/step - loss: 70.9575
Epoch 3/100
1/1 [==============================] - 0s 0s/step - loss: 70.9563
Epoch 4/100
1/1 [==============================] - 0s 0s/step - loss: 70.9551
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 70.9539
Epoch 6/100
1/1 [==============================] - 0s 16ms/step - loss: 70.9528
Epoch 7/100
1/1 [==============================] - 0s 17ms/step - loss: 70.9516
Epoch 8/100
1/1 [==============================] - 0s 0s/step - loss: 70.9504
Epoch 9/100
1/1 [==============================] - 0s 0s/step - loss: 70.9492
Epoch 10/100
1/1 [==============================] - 0s 17ms/step - loss: 70.9481
Epoch 11/100
1/1 [==============================] - 0s 18ms/step - loss: 70.9469
Epoch 12/100
1/1 [==============================] - 0s 0s/step - loss: 70.9457
Epoch 13/100
1/1 [=============================

In [32]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[-0.8972172,  1.2215649],
       [ 1.2616904, -0.930591 ]], dtype=float32), array([0.09999933, 0.09999933], dtype=float32)]
season        2001.397492
team_1        5593.078449
team_2        5616.694955
home             0.000000
seed_diff        0.053952
score_diff      -0.070283
score_1         70.934966
score_2         71.005249
won              0.500437
pred             0.123378
dtype: float64


In [33]:
# Evaluate the model on the tournament test data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']], games_tourney_test[['score_1', 'score_2']], verbose=False))

71.73503875732422


In [34]:
# Create an input layer with 2 columns
input_tensor = Input((2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear', use_bias=False)(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid', use_bias=False)(output_tensor_1)

# Create a model with 2 outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

In [35]:
# Import the Adam optimizer
from tensorflow.keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimzer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(learning_rate=0.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(games_tourney_train[['seed_diff', 'pred']],
          [games_tourney_train[['score_diff']], games_tourney_train[['won']]],
          epochs=10,
          verbose=True,
          batch_size=16384)

Epoch 1/10
1/1 [==============================] - 1s 604ms/step - loss: 12.7892 - dense_6_loss: 11.8458 - dense_7_loss: 0.9434
Epoch 2/10
1/1 [==============================] - 0s 11ms/step - loss: 12.6973 - dense_6_loss: 11.8074 - dense_7_loss: 0.8899
Epoch 3/10
1/1 [==============================] - 0s 14ms/step - loss: 12.6097 - dense_6_loss: 11.7690 - dense_7_loss: 0.8407
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 12.5265 - dense_6_loss: 11.7307 - dense_7_loss: 0.7959
Epoch 5/10
1/1 [==============================] - 0s 13ms/step - loss: 12.4479 - dense_6_loss: 11.6925 - dense_7_loss: 0.7555
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 12.3739 - dense_6_loss: 11.6545 - dense_7_loss: 0.7195
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 12.3045 - dense_6_loss: 11.6167 - dense_7_loss: 0.6878
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 12.2395 - dense_6_loss: 11.5792 - dense_7_loss

In [36]:
# Print the model weights
print(model.get_weights())

# Print the training data means
print(games_tourney_train.mean())

[array([[ 0.0435918],
       [-1.0518106]], dtype=float32), array([[1.6181574]], dtype=float32)]
season        2001.397492
team_1        5593.078449
team_2        5616.694955
home             0.000000
seed_diff        0.053952
score_diff      -0.070283
score_1         70.934966
score_2         71.005249
won              0.500437
pred             0.123378
dtype: float64


In [37]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = 0.14

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.5349429451582145
0.8021838885585818


In [38]:
# Evaluate the model on new data
print(model.evaluate(games_tourney_test[['seed_diff', 'pred']],
               [games_tourney_test[['score_diff']], games_tourney_test[['won']]], verbose=False))

[12.397934913635254, 11.788229942321777, 0.6097074747085571]
